In [17]:
from legisph.senate.website import SenateWebsite
from legisph.senate.models import SenateBill

senate = SenateWebsite()
bills = list(filter(lambda b: not isinstance(b, Exception), senate.fetch_all_bills()))


In [27]:
import polars as pl
from itertools import chain

all_statuses = [
    *[b.legislative_status for b in bills],
    *list(chain(*[b.legislative_history for b in bills])),
]
all_statuses = list(
    chain(
        *[
            [
                SenateBill.SenateBillStatus(date=status.date, item=s)
                for s in status.item.split(";\n")
            ]
            for status in all_statuses
        ]
    )
)

In [28]:
from legisph.senate.bill_status import *

unparsed_statuses = [
    status
    for status in all_statuses
    if len(
        parse_senate_bill_status(
            status,
            classes=[
                Introduced,
                FirstReading,
                OrganizationalBriefing,
                PendingInCommittee,
                CommitteeReportCalendaredForOrdinaryBusiness,
                ConsolidatedOrSubstitutedInCommitteeReport,
                TechnicalWorkingGroup,
                CommitteeProceedings,
                SubmittedCommitteeReport,
                PendingSecondReading,
                ApprovedOnSecondReading,
                TransferredToCalendarForSpecialOrder,
                PeriodOfInterpellationClosed,
                SponsorshipSpeech,
                SponsorshipSpeechInsertedIntoRecord,
                ApprovedOnThirdReading,
                SentToHouseOfRepresentatives,
                IndividualAmendmentsOpened,
                IndividualAmendmentsClosed,
                CommitteeAmendmentsOpened,
                CommitteeAmendmentsClosed,
                CopiesDistributed,
                ConsolidatedWithApprovedBill,
                ApprovedByPresident,
                ApprovedConferenceCommitteeReport,
                Interpellation,
                PendingInHouseOfRepresentatives,
                Withdrawn,
                Archived,
                Sponsored,
                VotesInFavor,
                VotesAgainst,
                VotesAbstained,
            ],
        )
    )
    == 0
]
status_df = pl.DataFrame({"status": [status.item for status in unparsed_statuses]})
list(
    (
        status_df.groupby("status")
        .agg([pl.count()])
        .sort("count", reverse=True)
        .filter(pl.col("count") > 10)
    )["status"]
)
# (
#     status_df.groupby("status")
#             .agg([pl.count()])
#             .sort("count", reverse=True)
#             .filter(pl.col("count") > 10)
# )


['Motion of the Majority Leader, Senator Alan Peter Compañero S. Cayetano, that the matters appearing in the Reference of Business for todays session be deemed read and transmitted to the Archives;',
 'Lapsed Into Law',
 'Period of interpellations closed;',
 'Approved on Second Reading with amendments;',
 'Transferred from the Committee on Constitutional Amendments, Revision of Codes and Laws to the Committee on ELECTORAL REFORMS  AND PEOPLES PARTICIPATION;',
 'Assigned for consideration by Special Order;',
 'Change of committee referral from the Committee on Youth, Women and Family Relations to the Committee on WOMEN, FAMILY RELATIONS AND GENDER EQUALITY;',
 'Conducted CONSULTATION/OCULAR;',
 'Period of amendments closed;',
 'Inquiry of the Chair;',
 'Approved on Third Reading',
 'Co-sponsorship speech of Senator LOREN B. LEGARDA;',
 'Returned and submitted by the Committee on WAYS AND MEANS per Committee Report No. 164, recommending that it be substituted by SBN-1592;',
 'SUBSTITUTED

In [5]:
from legisph.senate.bill_status import *
from legisph.senate.models import SenateBill
import datetime

matches = set([s.item for s in all_statuses if s.item.lower().find("against") > -1])
matches
# unparsed_statuses = [
#     status
#     for status in sponsorship_speech
#     if SponsorshipSpeech.parse(SenateBill.SenateBillStatus(
#         date = datetime.date.today(),
#         item = status
#     ))[0] is None
# ]
# unparsed_statuses


{'Against    : N o n e;',
 'Against  : (1) Senator Arroyo;',
 'Against  : (1) Senator Enrile;',
 'Against  : (2) Senators Lim and Pimentel, Jr.;',
 'Against:          (1) ANTONIO "SONNY" F. TRILLANES;',
 'Against:          N o n e;',
 'Against:         (1)  Arroyo;',
 'Against:         (1) JOKER P. ARROYO;',
 'Against:         (1) PIMENTEL, JR.;',
 'Against:         (2) JOKER P. ARROYO and FRANCIS "CHIZ" G. ESCUDERO;',
 'Against:         (2):  Senators LACSON and MADRIGAL;',
 'Against:         AQUILINO Q. PIMENTEL, JR.;',
 'Against:         N o n e ;',
 'Against:         N o n e;',
 'Against:        N o n e ;',
 'Against:        N o n e;',
 'Against:       (1) TEOFISTO "TG" GUINGONA III;',
 'Against:       N o n e;',
 'Against:      (1) JOKER P. ARROYO;',
 'Against:      N o n e;',
 'Against:   N o n e  ;',
 'Against:   N o n e ;',
 'Against:  (1) JOKER P. ARROYO;',
 'Against:  (1) MIRIAM DEFENSOR SANTIAGO;',
 'Against:  (1) RALPH G. RECTO;',
 'Against:  (1) RISA HONTIVEROS;',
 'Agains

In [1]:
a = list(
    filter(
        lambda b: "Consolidated/Substituted in the Committee Report"
        == b.legislative_status.item,
        bills,
    )
)
a[0]


NameError: name 'bills' is not defined

In [6]:
a = list(
    filter(
        lambda b: "Sponsor: Senator CHRISTOPHER LAWRENCE T. GO;"
        in [h.item for h in b.legislative_history],
        bills,
    )
)


In [11]:
a[0]


SenateBill(
    url="http://legacy.senate.gov.ph/lis/bill_res.aspx?q=SBN-2097&congress=18",
    congress=18,
    billno="SBN-2097",
    congress_text="18th Congress",
    billno_text="Senate Bill No. 2097",
    title="SUSPENSION OF PHILHEALTH PREMIUM CONTRIBUTION INCREASE",
    long_title="AN ACT GRANTING THE PRESIDENT OF THE PHILIPPINES THE POWER TO SUSPEND THE SCHEDULED INCREASES IN PHILHEALTH PREMIUM CONTRIBUTION RATES, AMENDING FOR THE PURPOSE REPUBLIC ACT NO. 11223,\nOTHERWISE KNOWN AS THE 'UNIVERSAL HEALTH CARE ACT'",
    filing_date=datetime.date(2021, 3, 10),
    filers=[
        Senator(name="Marcos, Imee R."),
        Senator(name="Poe, Grace"),
        Senator(name='Zubiri, Juan Miguel "Migz" F.'),
        Senator(name="Villanueva, Joel"),
        Senator(name="Angara, Sonny"),
        Senator(name="Binay, Maria Lourdes Nancy S."),
        Senator(name="Gatchalian, Win"),
        Senator(name="Gordon, Richard J."),
        Senator(name="Revilla Jr., Ramon Bong"),
        Sen

Cases to be handled: 
* items with newlines (\n) in them
* items with joint meanings but no newlines 
* senators names parsed correctly